# Deep Learning Homework: Image Classification 🧠📸

## Overview
You will build a deep learning model to classify medical images as either "tumor" or "normal" tissue. This homework covers the complete machine learning pipeline from data loading to model evaluation.

## Summary of All Questions

### **Question 1 (20 points) - Data Loading & Splitting**
- Load CSV file with image paths and labels
- Split data into train/test/validation sets (split of your choice). 
    - Split on the patient level (not the tile level). All tiles from the same patient should be in the same set.
- **Files to use**: Start with `sampled_tiles.csv`, then use `all_tiles.csv` for final submission
    - When using 'all_tiles.csv', will need to request a node from the cluster and run your code as a .py file using an sbatch script 
        - After testing your code with the smaller 'sampled_tiles.csv' file here, move your code to a .py file in the src folder and run it using an sbatch script 
        - I have provided an example sbatch script in the bash_scripts folder 

### **Question 2 (40 points) - Dataset & DataLoader**
- **Part A (20 points)**: Build `ImageDataset` class to load images and convert labels
- **Part B (20 points)**: Create DataLoaders with batch_size=32 and proper shuffling
- **Goal**: Convert text labels ('tumor'/'normal') to numbers (1/0)

### **Question 3 (20 points) - Model Training**
- Choose a model from `torchvision.models` 
- Implement training loop: forward pass → loss → backward pass → parameter update
- Add progress printing and model saving
- **Key concepts**: `optimizer.zero_grad()`, `loss.backward()`, `optimizer.step()`

### **Question 4 (20 points) - Model Evaluation**
- Evaluate trained model on test set
- Calculate and print test loss and accuracy
- Create ROC curve visualization
- **Goal**: Achieve at least 70% accuracy on test set

## Learning Objectives
By the end of this homework, you will understand:
- How to build custom PyTorch datasets
- The complete training loop in deep learning
- Model evaluation and visualization techniques
- Best practices for image classification


### **To submit in brightspace**
A zip file containing the following:
1. train_evaluate.py - the python file used to train the model, evaluate the model, and plot the ROC curve
2. training log file - training loss and accuracy for each epoch
3. slide level probabilities (for each slide in the test set. aggregate by taking the mean of the tile level probabilities)
4. slide level and tile level ROC curve plot


**Good luck! 🚀**


In [ ]:
# Import all necessary libraries
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.models as models
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import os

## Data Structure Example 📊

The CSV file has these important columns:
```
L1path,Tumor_Normal
/gpfs/data/courses/ml2025/data/image1.jpg,tumor
/gpfs/data/courses/ml2025/data/image2.jpg,normal
/gpfs/data/courses/ml2025/data/image3.jpg,tumor
...
```

**Column names:**
- `L1path`: Full path to the image file
- `Tumor_Normal`: Label ('tumor' or 'normal')

**Goal:** Convert 'tumor' → 1, 'normal' → 0 for your model


**Question 1** (20 points)

Write a function that:
1. Load the csv file with the tile paths and labels located here: ['/gpfs/data/courses/ml2025/labels/'] 
    - Start with `sampled_tiles.csv` (smaller dataset) for testing
    - Use `all_tiles.csv` for final submission
2. Split the tiles into train, test, and validation sets using a given train:test:validation ratio

In [ ]:
def load_split_data(csv_file, train_ratio, test_ratio, val_ratio):
    # TODO: load the csv file. 
    # Use either sampled_tiles.csv (use for testing on a small subset of tiles)
    # or all_tiles.csv (use for final submission)

    # TODO: split the tiles into train, test, and validation sets using the given train:test:validation ratio
    
    #return the train, test, and validation sets
    return train_set, test_set, val_set


**Question 2** (40 points)
1. Build a dataset class using the backbones provided (20 points)
2. Build a dataloader for the train, test, and validation sets (20 points)
    - BATCH_SIZE = 32
    - shuffle: set to True to have the data reshuffled at every epoch

In [ ]:

class ImageDataset(Dataset):
    def __init__(self, df, img_dir):
        """
        Custom dataset class for loading images and labels.
        
        Args:
            df: pandas DataFrame with columns to the full path of each tile ['L1path'] and the respective label ['Tumor_Normal'] 
            img_dir: folder where all your images are stored #/gpfs/data/courses/ml2025/data
        """
        self.df = df
        self.img_dir = img_dir
        # Map text labels to numbers: tumor=1, normal=0
        self.label_map = {'tumor': 1, 'normal': 0}  

    def __len__(self):
        # TODO: Return the total number of images in the dataset
        # Hint: How many rows are in self.df?
        return #FILL IN HERE

    def __getitem__(self, idx):
        """
        Get one image(tile) and its label using the index
        
        Args:
            idx: index of the image to retrieve
            
        Returns:
            image: tensor of the image
            label: integer label (0 or 1)
        """
        # TODO: Get the full path of the images from the dataframe
        
        # TODO: Get the text label from the dataframe  
       
        # TODO: Convert text label to number using label_map
      
        # TODO: Read the image file. Images should be tensors with shape [C, H, W] (channels, height, width)
    
        return image, label


In [ ]:

# TODO: Create DataLoaders for training, validation, and test sets
# Hint: You'll need to import DataLoader first: from torch.utils.data import DataLoader

# TODO: Create training DataLoader
train_loader = DataLoader()

# TODO: Create validation DataLoader  
val_loader = DataLoader()

# TODO: Create test DataLoader
test_loader = DataLoader()

Loop through one of the dataloaders to confirm the data is loaded correctly (this gives us the shape and label of one tile) :

In [ ]:
for images, label in train_loader:
    print("label: ", label)
    print("images.shape: ", images.shape)
    break

**Question 3** (20 points)
1. Create a function to train a model with your data
    - pick a model of your choice from torchvision.models
    - define a loss and optimizer and justify your choice
    -loop through each epoch, and for each batch of tiles, clear the gradients, forward pass, calculate the loss, backward pass, and update the model parameters
    -print the loss and epoch every few epochs

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    """
    Train the model for the specified number of epochs.
    
    Args:
        model: PyTorch model to train
        train_loader: DataLoader for training data
        criterion: loss function
        optimizer: optimizer
        num_epochs: number of training epochs
    """
    train_loss = []
    val_loss = []
    for epoch in range(num_epochs):
        epochtrain_loss = 0
        model.train()
        #loop through the tiles, and train the model on each batch of tiles
        for images, labels in train_loader:
            # TODO: Clear gradients from previous iteration
            # Hint: Use optimizer.zero_grad()
            
            # TODO: Forward pass - get model predictions
            # Hint: Pass images through the model
            
            # TODO: Calculate loss between predictions and true labels
            # Hint: Use criterion(model_outputs, labels (as a tensor))
            
            # TODO: Backward pass - compute gradients
            # Hint: Use loss.backward()
            
            # TODO: Update model parameters
            # Hint: Use optimizer.step()

            # TODO: Update epoch_train_loss
            # Hint: Moving average of the loss

        # TODO: Validation step at each epoch end
        # Hint: Use model.eval() to set the model to evaluation mode
        # Hint: Use torch.no_grad() to disable gradient computation
        # Hint: Inference with the entire validation set, record the loss and other metrics if implemented
        
        #TODO: Print training progress (train_loss, val_loss, epoch) for each epoch
        
        # TODO:log training progress
        # Hint: apppend the epoch_train_loss and epoch_val_loss to train_loss and val_loss
        
        #TODO: Update the best model
        # Hint: Save the model on epoch 0. For the following epochs, save the model if the current model has a lower validation loss
        # Hint: Use torch.save(model.state_dict(), save_path)
    
    #TODO: save a training log file
    # Hint: Save the list of train_loss and val_loss to a txt file


**Question 4** (20 points)
- Evaluate the model on the test set

**Instructions:**
Implement a function to evaluate your trained model on the test set. You need to:

1. **Set model to evaluation mode** - Use `model.eval()` to disable dropout and batch normalization updates
2. **Initialize variables** to track:
   - Total test loss
   - Number of correct predictions
   - Total number of samples

3. **Loop through test data** (use `torch.no_grad()` to disable gradient computation):
   - Get model predictions
   - Calculate loss for this batch
   - Get predicted class (use `torch.max()` on outputs)
   - Track correct predictions

4. **Calculate and print metrics:**
   - Test loss: average loss across all test samples
   - Test accuracy: percentage of correct predictions

**Goal:** Get your model to achieve at least 70% accuracy on the test set!

In [ ]:
def evaluate_model(model, test_loader, criterion):
    """
    Evaluate the trained model on test data and calculate accuracy.
    
    Args:
        model: trained PyTorch model
        test_loader: DataLoader for test data
        criterion: loss function
    """
    #Setting model to evaluation mode
    model.eval()

    #Initializing variables to track accuracy
    total_loss = 0
    num_correct_predictions = 0
    total_tiles = 0
    true_labels_all = []
    predicted_probabilities_all = []
    
    #Loop through test data without computing gradients
    with torch.no_grad():
        for images, labels in test_loader:
            # TODO: Get model predictions (outputs of the model)
            # Hint: Pass images through the model
        
            # TODO: Calculate loss for this batch
            # Hint: Use criterion(outputs, labels)
            
            # TODO: Get predicted class
            # Hint: Use torch.max to get which class/label has the highest score (has the highest probability of occuring (0 or 1))

            #Update tracking variables
            # Hint: Add to total_loss, num_correct_predictions, total_tiles
            total_loss += loss.item()
            num_correct_predictions += (predicted == labels).sum().item()
            total_tiles += len(labels)
            
            predicted_probabilities = torch.softmax(outputs, dim=1)
            true_labels_all.append(labels)
            predicted_probabilities_all.append(predicted_probabilities)
    
    # TODO: Calculate and print final metrics
    # Hint: Calculate average loss and accuracy percentage
    # Hint: Print "Test Loss: {avg_loss:.4f}" and "Test Accuracy: {accuracy:.2f}%"
    
    #return the true labels and predicted probabilities
    return true_labels_all, predicted_probabilities_all


In [ ]:
def plot_roc_curve(true_labels_all, predicted_probabilities_all):
    """
    Create a simple ROC curve plot.
    
    Args:
        true_labels: numpy array of true binary labels (0 or 1)
        predicted_probabilities: numpy array of predicted probabilities for positive class
    """
    from sklearn.metrics import roc_curve, auc
    
    # TODO: Calculate ROC curve

    # TODO: Calculate AUC score

    # TODO: Plot the ROC curve
    
    pass  # Remove this line when you implement the function


In [ ]:
def main():
    #run all the functions above
    train_model(model, train_loader, criterion, optimizer, num_epochs=5)
    true_labels, predicted_probabilities = evaluate_model(model, test_loader, criterion)
    plot_roc_curve(true_labels, predicted_probabilities)

if __name__ == "__main__":
    main()